# WAVES

In [1]:
import pyodbc
import pandas as pd

In [2]:
cnxn = pyodbc.connect("Driver={SQL Server Native Client 11.0};"
                        "Server=wd02;"
                        "Database=FootPrint;"
                        "Trusted_Connection=yes;")

### Query

In [3]:
df = pd.read_sql_query("SELECT CONVERT(varchar,w.releaseDate,100) Date, w.id, wa.name Warehouse, s.lookupCode Shipments, p.name Customer, ss.name ShipmentStatus\
    FROM datex_footprint.Waves w\
    JOIN datex_footprint.WaveStatuses ws\
        ON ws.id = w.statusId\
    JOIN datex_footprint.Warehouses wa\
        ON wa.id = w.warehouseId\
    JOIN datex_footprint.shipments s\
        ON s.waveId = w.id\
    JOIN datex_footprint.ShipmentOrderLookup sol\
        ON sol.shipmentId = s.id\
    JOIN datex_footprint.Orders o\
        ON o.id = sol.orderId\
    JOIN datex_footprint.Projects p\
        ON p.id = o.projectId\
    JOIN datex_footprint.ShipmentStatuses ss\
        ON ss.id = s.statusId\
    WHERE releaseDate > dateadd(day,-1,getdate())\
        AND w.statusId = 3\
    ORDER BY releaseDate DESC", cnxn)

# Create Waves Table Data

In [4]:
df.head(10)

,Date,id,Warehouse,Shipments,Customer,ShipmentStatus
0,Aug 4 2022 8:30AM,243410,10,0085923027,ISDIN Corporation,Released
1,Aug 4 2022 8:30AM,243410,10,0085923023,ISDIN Corporation,Released
2,Aug 4 2022 8:30AM,243410,10,0085923024,ISDIN Corporation,Released
3,Aug 4 2022 8:30AM,243410,10,0085923029,ISDIN Corporation,Released
4,Aug 4 2022 8:30AM,243410,10,0085923028,ISDIN Corporation,Released
5,Aug 4 2022 8:30AM,243410,10,0085923031,ISDIN Corporation,Released
6,Aug 4 2022 8:30AM,243410,10,0085923026,ISDIN Corporation,Released
7,Aug 4 2022 8:30AM,243410,10,0085923025,ISDIN Corporation,Released
8,Aug 4 2022 8:30AM,243410,10,0085923042,ISDIN Corporation,Released
9,Aug 4 2022 8:30AM,243410,10,0085923058,ISDIN Corporation,Released


In [5]:
# Count Shipments per Wave and Status
count_shipments_statusdf = df.groupby(['id', 'ShipmentStatus']).count()[['Shipments']]
count_shipments_statusdf.reset_index(inplace=True)
count_shipments_statusdf.tail()

,id,ShipmentStatus,Shipments
27,243403,Released,13
28,243404,Executing,1
29,243406,Executing,1
30,243410,Released,19
31,243411,Released,11


In [6]:
# Convert Statuses to Columns
shipment_pivot = count_shipments_statusdf.pivot(index='id', columns='ShipmentStatus', values='Shipments')
shipment_pivot.tail()

ShipmentStatus,Completed,Executing,Released
id,,,
243403,NaN,NaN,13.0
243404,NaN,1.0,NaN
243406,NaN,1.0,NaN
243410,NaN,NaN,19.0
243411,NaN,NaN,11.0


In [7]:
# Convert NaN to 0
shipment_pivot = shipment_pivot.fillna(0)
shipment_pivot.head()

ShipmentStatus,Completed,Executing,Released
id,,,
243306,0.0,6.0,1.0
243316,0.0,0.0,1.0
243320,0.0,0.0,1.0
243321,0.0,0.0,1.0
243322,0.0,0.0,1.0


In [8]:
# Convert Float to Int
try:
    shipment_pivot['Completed'] = shipment_pivot['Completed'].astype(int)
except:
    print('Column Completed does not exist yet')

try:
    shipment_pivot['Executing'] = shipment_pivot['Executing'].astype(int)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'] = shipment_pivot['Released'].astype(int)
except:
    print('Column Released does not exist yet')

shipment_pivot.head()

ShipmentStatus,Completed,Executing,Released
id,,,
243306,0,6,1
243316,0,0,1
243320,0,0,1
243321,0,0,1
243322,0,0,1


In [9]:
# Convert id index to a column
shipment_pivot.reset_index(inplace=True)
shipment_pivot.head()

ShipmentStatus,id,Completed,Executing,Released
0,243306,0,6,1
1,243316,0,0,1
2,243320,0,0,1
3,243321,0,0,1
4,243322,0,0,1


In [10]:
# Convert zeros to blanks. This also change the data type to String
try:
    shipment_pivot['Completed'].replace([0, ''], '', inplace=True)
except:
    print('Column Completed does not exist yet')
    
try:
    shipment_pivot['Executing'].replace([0, ''], '', inplace=True)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'].replace([0, ''], '', inplace=True)
except:
    print('Column Released does not exist yet')
    
shipment_pivot.head()

ShipmentStatus,id,Completed,Executing,Released
0,243306,,6,1
1,243316,,,1
2,243320,,,1
3,243321,,,1
4,243322,,,1


In [11]:
shipment_pivot.dtypes

ShipmentStatus
id            int64
Completed    object
Executing    object
Released     object
dtype: object

In [12]:
# Change columns order
df = df[['Date', 'id', 'Warehouse', 'Customer']]

In [13]:
# Drops duplicates
df = df.drop_duplicates()

In [14]:
df.head(10)

,Date,id,Warehouse,Customer
0,Aug 4 2022 8:30AM,243410,10,ISDIN Corporation
19,Aug 4 2022 8:30AM,243411,10,ISDIN Corporation
30,Aug 3 2022 3:36PM,243406,12,Doctors Scientific Organica
31,Aug 3 2022 3:25PM,243404,12,Hush Hush
32,Aug 3 2022 3:24PM,243403,20,Alumier Labs Inc.
45,Aug 3 2022 2:50PM,243402,19,ACG North America LLC
48,Aug 3 2022 2:35PM,243396,10,ISDIN Corporation
53,Aug 3 2022 2:32PM,243395,10,ISDIN Corporation
62,Aug 3 2022 2:30PM,243385,20,"ACI Healthcare USA, Inc."
65,Aug 3 2022 1:29PM,243389,10,ISDIN Corporation


In [15]:
# Merge df with pivot table
df2 = df.merge(shipment_pivot, left_on='id', right_on='id')
df2.head(10)

,Date,id,Warehouse,Customer,Completed,Executing,Released
0,Aug 4 2022 8:30AM,243410,10,ISDIN Corporation,,,19
1,Aug 4 2022 8:30AM,243411,10,ISDIN Corporation,,,11
2,Aug 3 2022 3:36PM,243406,12,Doctors Scientific Organica,,1,
3,Aug 3 2022 3:25PM,243404,12,Hush Hush,,1,
4,Aug 3 2022 3:24PM,243403,20,Alumier Labs Inc.,,,13
5,Aug 3 2022 2:50PM,243402,19,ACG North America LLC,,2,1
6,Aug 3 2022 2:35PM,243396,10,ISDIN Corporation,,,5
7,Aug 3 2022 2:32PM,243395,10,ISDIN Corporation,,9,
8,Aug 3 2022 2:30PM,243385,20,"ACI Healthcare USA, Inc.",,2,1
9,Aug 3 2022 1:29PM,243389,10,ISDIN Corporation,,,7


In [16]:
wave_df = pd.DataFrame()
statuses = list(df2.columns)

# Reorder columns
if ('Released' in statuses) and ('Executing' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing', 'Completed']]
elif ('Released' in statuses) and ('Executing' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing']]
elif ('Released' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Completed']]
elif ('Released' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released']]
else:
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer']]

In [17]:
wave_df.head(10)

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Aug 4 2022 8:30AM,243410,10,ISDIN Corporation,19,,
1,Aug 4 2022 8:30AM,243411,10,ISDIN Corporation,11,,
2,Aug 3 2022 3:36PM,243406,12,Doctors Scientific Organica,,1,
3,Aug 3 2022 3:25PM,243404,12,Hush Hush,,1,
4,Aug 3 2022 3:24PM,243403,20,Alumier Labs Inc.,13,,
5,Aug 3 2022 2:50PM,243402,19,ACG North America LLC,1,2,
6,Aug 3 2022 2:35PM,243396,10,ISDIN Corporation,5,,
7,Aug 3 2022 2:32PM,243395,10,ISDIN Corporation,,9,
8,Aug 3 2022 2:30PM,243385,20,"ACI Healthcare USA, Inc.",1,2,
9,Aug 3 2022 1:29PM,243389,10,ISDIN Corporation,7,,


In [18]:
# Save Waves DF into json file
wave_df.to_json('data/waves.json')

# Warehouse Pie Chart Data

In [19]:
# Save df2 in df3
warehouse_df = wave_df
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Aug 4 2022 8:30AM,243410,10,ISDIN Corporation,19,,
1,Aug 4 2022 8:30AM,243411,10,ISDIN Corporation,11,,
2,Aug 3 2022 3:36PM,243406,12,Doctors Scientific Organica,,1,
3,Aug 3 2022 3:25PM,243404,12,Hush Hush,,1,
4,Aug 3 2022 3:24PM,243403,20,Alumier Labs Inc.,13,,


In [20]:
# Conver String to Float
try:
    warehouse_df['Released'] = pd.to_numeric(warehouse_df['Released'],errors='coerce')
except:
    print('Column Released does not exist yet')

try:
    warehouse_df['Executing'] = pd.to_numeric(warehouse_df['Executing'],errors='coerce')
except:
    print('Column Executing does not exist yet')

try:
    warehouse_df['Completed'] = pd.to_numeric(warehouse_df['Completed'],errors='coerce')
except:
    print('Column Completed does not exist yet')

warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Aug 4 2022 8:30AM,243410,10,ISDIN Corporation,19.0,NaN,NaN
1,Aug 4 2022 8:30AM,243411,10,ISDIN Corporation,11.0,NaN,NaN
2,Aug 3 2022 3:36PM,243406,12,Doctors Scientific Organica,NaN,1.0,NaN
3,Aug 3 2022 3:25PM,243404,12,Hush Hush,NaN,1.0,NaN
4,Aug 3 2022 3:24PM,243403,20,Alumier Labs Inc.,13.0,NaN,NaN


In [21]:
# Conver NaN to Zero
warehouse_df = warehouse_df.fillna(0)
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Aug 4 2022 8:30AM,243410,10,ISDIN Corporation,19.0,0.0,0.0
1,Aug 4 2022 8:30AM,243411,10,ISDIN Corporation,11.0,0.0,0.0
2,Aug 3 2022 3:36PM,243406,12,Doctors Scientific Organica,0.0,1.0,0.0
3,Aug 3 2022 3:25PM,243404,12,Hush Hush,0.0,1.0,0.0
4,Aug 3 2022 3:24PM,243403,20,Alumier Labs Inc.,13.0,0.0,0.0


In [22]:
# Group by Warehouse
warehouse_df = warehouse_df.groupby(['Warehouse']).sum()
warehouse_df

,id,Released,Executing,Completed
Warehouse,,,,
10,2433786,63.0,9.0,0.0
12,730172,1.0,2.0,0.0
15,243306,1.0,6.0,0.0
16,243354,0.0,1.0,0.0
19,730106,2.0,5.0,0.0
20,973544,21.0,7.0,7.0
21,729963,3.0,0.0,0.0


In [23]:
# Reset Index
warehouse_df.reset_index(inplace=True)
warehouse_df

,Warehouse,id,Released,Executing,Completed
0,10,2433786,63.0,9.0,0.0
1,12,730172,1.0,2.0,0.0
2,15,243306,1.0,6.0,0.0
3,16,243354,0.0,1.0,0.0
4,19,730106,2.0,5.0,0.0
5,20,973544,21.0,7.0,7.0
6,21,729963,3.0,0.0,0.0


In [24]:
warehouses = list(warehouse_df.columns)
warehouse_df2 = pd.DataFrame()

if ('Released' in warehouses) and ('Executing' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing', 'Completed']]
elif ('Released' in warehouses) and ('Executing' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing']]
elif ('Released' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Completed']]
elif ('Released' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released']]
else:
    warehouse_df2 = warehouse_df[['Warehouse']]

warehouse_df2

,Warehouse,Released,Executing,Completed
0,10,63.0,9.0,0.0
1,12,1.0,2.0,0.0
2,15,1.0,6.0,0.0
3,16,0.0,1.0,0.0
4,19,2.0,5.0,0.0
5,20,21.0,7.0,7.0
6,21,3.0,0.0,0.0


In [25]:
warehouse_df2 = pd.melt(warehouse_df2, id_vars=["Warehouse"], 
                  var_name="Status", value_name="Value")
warehouse_df2.head()

,Warehouse,Status,Value
0,10,Released,63.0
1,12,Released,1.0
2,15,Released,1.0
3,16,Released,0.0
4,19,Released,2.0


In [26]:
warehouse_df3 = warehouse_df2.pivot(index ='Status', columns='Warehouse', values='Value')
warehouse_df3

Warehouse,10,12,15,16,19,20,21
Status,,,,,,,
Completed,0.0,0.0,0.0,0.0,0.0,7.0,0.0
Executing,9.0,2.0,6.0,1.0,5.0,7.0,0.0
Released,63.0,1.0,1.0,0.0,2.0,21.0,3.0


In [27]:
warehouse_df4 = warehouse_df3.reset_index()
warehouse_df4

Warehouse,Status,10,12,15,16,19,20,21
0,Completed,0.0,0.0,0.0,0.0,0.0,7.0,0.0
1,Executing,9.0,2.0,6.0,1.0,5.0,7.0,0.0
2,Released,63.0,1.0,1.0,0.0,2.0,21.0,3.0


In [28]:
warehouse_df4 = warehouse_df4.rename_axis(None, axis=1)
warehouse_df4

,Status,10,12,15,16,19,20,21
0,Completed,0.0,0.0,0.0,0.0,0.0,7.0,0.0
1,Executing,9.0,2.0,6.0,1.0,5.0,7.0,0.0
2,Released,63.0,1.0,1.0,0.0,2.0,21.0,3.0


In [29]:
# Save Warehouse into JSON file
warehouse_df4.to_json('data/warehouse.json')

In [30]:
# Close connection
cnxn.close()